In [83]:
import re
import os
import time
import regex
import emoji
import pandas as pd
import numpy as np
import preprocessor as p
import joblib

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from corextopic import corextopic as ct

In [4]:
resource_dir = '/home/ed/github/pod_tweets/resources/'
tweets_dir = '/home/ed/github/pod_tweets/follower_twts/'

p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.EMOJI)
punct_str = '''!"$%&'()*+,-./:;<=>?[\]^_`{|}~’'''
# stop_words = ['the']

In [5]:
stop_words = [# 'ad',
    'am',
    'an',
    'as',
    'at',
    'be',
    'by',
    # 'dc',
    'de',
    'do',
    # 'dr',
    'el',
    'en',
    'go',
    'he',
    'hi',
    'id',
    'if',
    'im',
    'in',
    'is',
    'it',
    'la',
    'me',
    # 'mr',
    'my',
    'no',
    'of',
    'oh',
    'ok',
    'on',
    'or',
    'pm',
    'rt',
    'so',
    'st',
    'to',
    # 'tv',
    'up',
    'us',
    # 'vs',
    'we',
    'ya',
    '1st',
    # 'act',
    # 'age',
    # 'ago',
    # 'air',
    'all',
    'amp',
    'and',
    'any',
    # 'app',
    'are',
    # 'art',
    'ask',
    # 'ass',
    # 'bad',
    'big',
    'bit',
    'boy',
    'but',
    'buy',
    'can',
    'car',
    'dad',
    'day',
    'did',
    'die',
    'dog',
    'due',
    'end',
    'etc',
    # 'fan',
    'far',
    'few',
    'for',
    # 'fox',
    # 'fun',
    'get',
    # 'god',
    # 'gop',
    'got',
    # 'gun',
    'guy',
    'had',
    'has',
    'her',
    'hes',
    'hey',
    'him',
    'his',
    # 'hit',
    'how',
    'ill',
    'its',
    'ive',
    'jim',
    'job',
    'joe',
    # 'key',
    # 'kid',
    'law',
    'let',
    'lie',
    'lol',
    'lot',
    'low',
    'man',
    'map',
    'may',
    # 'men',
    'met',
    # 'mom',
    # 'new',
    # 'nfl',
    'not',
    'now',
    'off',
    # 'old',
    'omg',
    'one',
    'our',
    'out',
    'own',
    # 'pay',
    'put',
    'que',
    'red',
    'run',
    # 'sad',
    'saw',
    'say',
    'see',
    'set',
    'she',
    'sir',
    # 'son',
    # 'tax',
    'the',
    'too',
    'top',
    'try',
    'two',
    'use',
    'via',
    # 'war',
    'was',
    'way',
    'who',
    'why',
    # 'win',
    # 'won',
    'wow',
    'yes',
    'yet',
    'you',
    # '2020',
    # 'able',
    # 'also',
    # 'away',
    # 'baby',
    # 'back',
    # 'beat',
    # 'been',
    # 'best',
    # 'bill',
    # 'blue',
    # 'book',
    # 'born',
    # 'both',
    # 'bowl',
    # 'call',
    # 'came',
    # 'cant',
    # 'care',
    # 'case',
    # 'city',
    # 'code',
    # 'cold',
    # 'come',
    # 'cool',
    # 'cost',
    'damn',
    # 'data',
    # 'date',
    # 'days',
    # 'dead',
    # 'deal',
    # 'dear',
    # 'dems',
    # 'died',
    # 'does',
    # 'done',
    # 'dont',
    # 'down',
    # 'each',
    # 'easy',
    # 'else',
    # 'even',
    # 'ever',
    # 'face',
    # 'fact',
    # 'fair',
    # 'fake',
    # 'fall',
    # 'fans',
    # 'fear',
    # 'feel',
    # 'film',
    # 'find',
    # 'fire',
    # 'food',
    # 'four',
    # 'free',
    # 'from',
    'fuck',
    # 'full',
    # 'game',
    # 'gave',
    # 'gets',
    # 'gift',
    # 'girl',
    # 'give',
    # 'glad',
    # 'goes',
    # 'good',
    # 'guys',
    # 'half',
    # 'hand',
    # 'hard',
    # 'hate',
    'have',
    # 'head',
    # 'hear',
    # 'hell',
    # 'help',
    # 'here',
    # 'high',
    # 'hold',
    # 'home',
    # 'hope',
    # 'hour',
    # 'huge',
    # 'idea',
    # 'info',
    # 'into',
    # 'iowa',
    # 'iran',
    # 'isnt',
    # 'jobs',
    # 'john',
    # 'join',
    # 'just',
    # 'keep',
    # 'kids',
    # 'kind',
    # 'king',
    # 'knew',
    # 'know',
    # 'kobe',
    # 'lack',
    # 'last',
    # 'late',
    # 'lead',
    # 'left',
    # 'less',
    # 'lets',
    # 'lies',
    # 'life',
    # 'line',
    # 'link',
    # 'list',
    # 'live',
    # 'long',
    # 'look',
    # 'lose',
    # 'lost',
    # 'lots',
    # 'love',
    # 'luck',
    # 'made',
    # 'make',
    # 'many',
    # 'mean',
    # 'meet',
    # 'mike',
    # 'mind',
    # 'miss',
    # 'more',
    # 'most',
    # 'move',
    # 'much',
    # 'must',
    # 'name',
    # 'near',
    # 'need',
    # 'news',
    # 'next',
    # 'nice',
    # 'okay',
    # 'once',
    # 'ones',
    # 'only',
    # 'open',
    # 'over',
    # 'paid',
    # 'park',
    # 'part',
    # 'pass',
    # 'past',
    # 'pete',
    # 'pick',
    # 'plan',
    # 'play',
    # 'poor',
    # 'post',
    # 'race',
    # 'read',
    # 'real',
    # 'rest',
    # 'room',
    # 'rule',
    # 'rush',
    # 'safe',
    # 'said',
    # 'same',
    # 'save',
    # 'says',
    # 'seem',
    # 'seen',
    # 'send',
    # 'sent',
    # 'shes',
    # 'shit',
    # 'shot',
    # 'show',
    # 'sick',
    # 'side',
    # 'sign',
    # 'some',
    # 'song',
    # 'soon',
    # 'star',
    # 'stay',
    # 'step',
    # 'stop',
    # 'such',
    # 'sure',
    # 'take',
    # 'talk',
    # 'team',
    # 'tell',
    # 'than',
    # 'that',
    # 'them',
    # 'then',
    # 'they',
    # 'time',
    # 'told',
    # 'took',
    # 'true',
    # 'turn',
    # 'used',
    # 'very',
    # 'vote',
    # 'wait',
    # 'want',
    # 'ways',
    # 'week',
    # 'well',
    # 'went',
    # 'were',
    # 'weve',
    # 'what',
    # 'when',
    # 'whos',
    # 'wife',
    # 'will',
    # 'wins',
    # 'wish',
    # 'with',
    # 'wont',
    # 'word',
    # 'work',
    # 'yall',
    # 'yeah',
    # 'year',
    # 'york',
    'your',
    # 'about',
    # 'after',
    # 'again',
    # 'agree',
    # 'allow',
    # 'alone',
    # 'along',
    # 'among',
    # 'apply',
    # 'arent',
    # 'asked',
    # 'based',
    # 'being',
    # 'biden',
    # 'black',
    # 'board',
    # 'books',
    # 'brain',
    # 'break',
    # 'bring',
    # 'build',
    # 'calls',
    # 'cause',
    # 'check',
    # 'child',
    # 'china',
    # 'class',
    # 'clear',
    # 'close',
    # 'comes',
    # 'could',
    # 'court',
    # 'cover',
    # 'crazy',
    # 'daily',
    # 'death',
    # 'didnt',
    # 'doing',
    # 'early',
    # 'email',
    # 'enjoy',
    # 'event',
    # 'every',
    # 'facts',
    # 'field',
    # 'fight',
    # 'final',
    # 'first',
    # 'folks',
    # 'force',
    # 'found',
    # 'front',
    # 'funny',
    # 'games',
    # 'girls',
    # 'given',
    # 'going',
    # 'gonna',
    # 'great',
    # 'green',
    # 'group',
    # 'guess',
    # 'hands',
    # 'happy',
    # 'heard',
    # 'heart',
    # 'hello',
    # 'heres',
    # 'honor',
    # 'hours',
    # 'house',
    # 'human',
    # 'idiot',
    # 'issue',
    # 'jesus',
    # 'knows',
    # 'later',
    # 'learn',
    # 'least',
    # 'leave',
    # 'legal',
    # 'level',
    # 'light',
    # 'lives',
    # 'local',
    # 'looks',
    # 'loved',
    # 'makes',
    # 'march',
    # 'maybe',
    # 'means',
    # 'media',
    # 'might',
    # 'mitch',
    # 'money',
    # 'month',
    # 'moved',
    # 'movie',
    # 'music',
    # 'needs',
    # 'never',
    # 'night',
    # 'obama',
    # 'often',
    # 'order',
    # 'other',
    # 'party',
    # 'peace',
    # 'phone',
    # 'photo',
    # 'piece',
    # 'place',
    # 'point',
    # 'power',
    # 'press',
    # 'proud',
    # 'radio',
    # 'reach',
    # 'ready',
    # 'right',
    # 'rules',
    # 'seems',
    # 'sense',
    # 'shame',
    # 'share',
    # 'short',
    # 'shows',
    # 'since',
    # 'small',
    # 'sorry',
    # 'south',
    # 'space',
    # 'speak',
    # 'spend',
    # 'spent',
    # 'staff',
    # 'stage',
    # 'stand',
    # 'start',
    # 'state',
    # 'still',
    # 'story',
    # 'stuff',
    # 'super',
    # 'takes',
    # 'texas',
    # 'thank',
    'thats',
    'their',
    'there',
    'these',
    'thing',
    'think',
    'those',
    # 'three',
    # 'times',
    # 'tired',
    # 'today',
    # 'trial',
    # 'tried',
    # 'truly',
    # 'trump',
    # 'truth',
    # 'tweet',
    'u200d',
    # 'under',
    # 'union',
    # 'until',
    # 'using',
    # 'video',
    # 'views',
    # 'visit',
    # 'voice',
    # 'voted',
    # 'voter',
    # 'votes',
    # 'wants',
    # 'wasnt',
    # 'watch',
    # 'water',
    # 'weeks',
    # 'weird',
    'whats',
    'where',
    'which',
    'while',
    # 'white',
    # 'whole',
    # 'woman',
    # 'women',
    # 'words',
    # 'works',
    # 'world',
    # 'worst',
    # 'worth',
    # 'would',
    # 'write',
    # 'wrong',
    # 'wrote',
    # 'years',
    'youll',
    # 'young',
    'youre',
    'yours',
    'youve',
    # 'access',
    # 'across',
    # 'action',
    # 'almost',
    # 'always',
    # 'answer',
    # 'anthem',
    # 'anyone',
    # 'around',
    # 'asking',
    # 'attack',
    # 'become',
    # 'before',
    # 'behind',
    # 'bernie',
    # 'better',
    # 'beyond',
    # 'bolton',
    # 'bryant',
    # 'called',
    # 'cancer',
    # 'cannot',
    # 'career',
    # 'caucus',
    # 'center',
    # 'chance',
    # 'change',
    # 'chiefs',
    # 'choice',
    # 'choose',
    # 'church',
    # 'coming',
    # 'county',
    # 'course',
    # 'create',
    # 'crisis',
    # 'debate',
    # 'doesnt',
    # 'donald',
    # 'donate',
    # 'during',
    # 'either',
    # 'energy',
    # 'enough',
    # 'entire',
    # 'expect',
    # 'failed',
    # 'family',
    # 'follow',
    # 'forget',
    # 'former',
    # 'friday',
    # 'friend',
    # 'future',
    # 'giving',
    # 'global',
    # 'google',
    # 'happen',
    # 'havent',
    # 'having',
    # 'health',
    # 'impact',
    # 'issues',
    # 'joined',
    # 'kansas',
    # 'killed',
    # 'latest',
    # 'leader',
    # 'likely',
    # 'listen',
    # 'little',
    # 'living',
    # 'longer',
    # 'making',
    # 'matter',
    # 'member',
    # 'middle',
    # 'missed',
    # 'moment',
    # 'monday',
    # 'months',
    # 'myself',
    # 'nation',
    # 'needed',
    # 'number',
    # 'office',
    # 'online',
    # 'others',
    # 'paying',
    # 'people',
    # 'person',
    # 'please',
    # 'points',
    # 'police',
    # 'policy',
    # 'pretty',
    # 'prison',
    # 'public',
    # 'racism',
    # 'racist',
    # 'rather',
    # 'really',
    # 'reason',
    # 'record',
    # 'remove',
    # 'report',
    # 'review',
    # 'rights',
    # 'russia',
    # 'saying',
    # 'school',
    # 'season',
    # 'second',
    # 'seeing',
    # 'senate',
    # 'series',
    # 'sexual',
    # 'should',
    # 'single',
    # 'social',
    # 'sounds',
    # 'speech',
    # 'sports',
    # 'spread',
    # 'states',
    # 'street',
    # 'strong',
    # 'stupid',
    # 'sunday',
    # 'system',
    # 'taking',
    # 'thanks',
    'theres',
    'theyre',
    # 'things',
    # 'thinks',
    # 'though',
    # 'thread',
    # 'todays',
    # 'trumps',
    # 'trying',
    # 'tweets',
    # 'united',
    # 'update',
    # 'voters',
    # 'voting',
    # 'wanted',
    # 'warren',
    # 'wonder',
    # 'worked',
    # 'account',
    # 'address',
    # 'against',
    # 'already',
    # 'amazing',
    # 'america',
    # 'another',
    # 'article',
    # 'awesome',
    # 'because',
    # 'believe',
    # 'between',
    # 'biggest',
    # 'billion',
    # 'brought',
    # 'calling',
    # 'changed',
    # 'climate',
    # 'clinton',
    # 'college',
    # 'comment',
    # 'company',
    # 'content',
    # 'control',
    # 'couldnt',
    # 'country',
    # 'culture',
    # 'current',
    # 'defense',
    # 'deserve',
    # 'elected',
    # 'episode',
    # 'exactly',
    # 'excited',
    # 'federal',
    # 'feeling',
    # 'finally',
    # 'florida',
    # 'foreign',
    # 'forever',
    # 'forward',
    # 'friends',
    # 'fucking',
    # 'general',
    # 'getting',
    # 'happens',
    # 'hearing',
    # 'hillary',
    # 'himself',
    # 'history',
    # 'imagine',
    # 'instead',
    # 'january',
    # 'justice',
    # 'leaders',
    # 'looking',
    # 'matters',
    # 'medical',
    # 'meeting',
    # 'members',
    # 'message',
    # 'michael',
    # 'million',
    # 'minutes',
    # 'morning',
    # 'nothing',
    # 'opinion',
    # 'outside',
    # 'parents',
    # 'perfect',
    # 'picture',
    # 'playing',
    # 'podcast',
    # 'primary',
    # 'problem',
    # 'process',
    # 'program',
    # 'project',
    # 'protect',
    # 'putting',
    # 'reading',
    # 'realize',
    # 'respect',
    # 'results',
    # 'retweet',
    # 'running',
    # 'sanders',
    # 'schools',
    # 'science',
    # 'senator',
    # 'serious',
    # 'service',
    # 'sharing',
    # 'showing',
    # 'sitting',
    # 'someone',
    # 'special',
    # 'started',
    # 'stories',
    # 'student',
    # 'support',
    # 'talking',
    # 'telling',
    # 'thought',
    # 'through',
    # 'tonight',
    # 'totally',
    # 'tuesday',
    # 'twitter',
    # 'ukraine',
    # 'waiting',
    # 'watched',
    # 'weekend',
    # 'welcome',
    # 'whether',
    # 'winning',
    # 'without',
    # 'workers',
    # 'working',
    # 'wouldnt',
    # 'writing',
    # 'actually',
    # 'american',
    # 'anything',
    # 'argument',
    # 'behavior',
    # 'birthday',
    # 'breaking',
    # 'building',
    # 'business',
    # 'campaign',
    # 'caucuses',
    # 'children',
    # 'congrats',
    # 'congress',
    # 'continue',
    # 'criminal',
    # 'daughter',
    # 'decision',
    # 'democrat',
    # 'election',
    # 'everyone',
    # 'evidence',
    # 'families',
    # 'favorite',
    # 'fighting',
    # 'football',
    # 'halftime',
    # 'happened',
    # 'interest',
    # 'language',
    # 'learning',
    # 'majority',
    # 'military',
    # 'millions',
    # 'missouri',
    # 'national',
    # 'november',
    # 'official',
    # 'personal',
    # 'politics',
    # 'possible',
    # 'powerful',
    # 'probably',
    # 'question',
    # 'remember',
    # 'reminder',
    # 'research',
    # 'response',
    # 'saturday',
    # 'security',
    # 'senators',
    # 'shouldnt',
    # 'starting',
    # 'students',
    # 'thinking',
    # 'thoughts',
    # 'together',
    # 'tomorrow',
    # 'training',
    # 'violence',
    # 'virginia',
    # 'watching',
    # 'whatever',
    # 'yourself',
    # 'americans',
    # 'attention',
    # 'available',
    # 'beautiful',
    # 'bloomberg',
    # 'candidate',
    # 'celebrate',
    # 'christmas',
    # 'community',
    # 'democracy',
    # 'democrats',
    # 'different',
    # 'documents',
    # 'education',
    # 'following',
    # 'happening',
    # 'impeached',
    # 'important',
    # 'including',
    # 'interview',
    # 'listening',
    # 'literally',
    # 'mcconnell',
    # 'political',
    # 'president',
    # 'questions',
    # 'seriously',
    # 'something',
    # 'sometimes',
    # 'statement',
    # 'witnesses',
    # 'wonderful',
    # 'yesterday',
    # 'absolutely',
    # 'candidates',
    # 'commercial',
    # 'completely',
    # 'conference',
    # 'corruption',
    # 'definitely',
    # 'difference',
    # 'democratic',
    'especially',
    # 'everything',
    # 'experience',
    # 'government',
    # 'healthcare',
    # 'incredible',
    # 'leadership',
    # 'presidents',
    # 'republican',
    # 'supporters',
    # 'supporting',
    'themselves',
    'u2066u2069',
    # 'understand',
    # 'university',
    # 'washington',
    # 'communities',
    # 'coronavirus',
    # 'immigration',
    # 'impeachment',
    # 'information',
    # 'interesting',
    # 'opportunity',
    # 'republicans',
    # 'constitution',
    # 'conversation',
    # 'presidential',
    # 'administration',
    # 'congratulations'
]

In [6]:
def find_emoji(text):
    '''
    '''
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list


def find_hash(text):
    '''
    '''
    hashtag_list = []
    data = regex.findall(r'#\w*', text)
    for word in data:
        hashtag_list.append(word)
    return hashtag_list


def find_mention(text):
    '''
    '''
    mention_list = []
    data = regex.findall(r'@\w*', text)
    for word in data:
        mention_list.append(word)
    return mention_list


def tweet_split_2_process(user_id, tweets_str):
    '''
    '''
    df_out = pd.DataFrame(columns=['user_id'], dtype=np.int32)
    output_txt = ''
    output_emoji = ''
    output_hash = ''
    output_mention = ''
    
    tweets_str = tweets_str.replace('''', "''', "', '")
    tweets_str = tweets_str.replace('''", ''''', "', '")
    tweets_str = tweets_str.replace('''", "''', "', '")
    tweet_list = tweets_str.split("', '")
    
    for tweet in tweet_list:
        tweet_p = my_preprocess(tweet)
        output_txt = output_txt + ' ' + tweet_p[0]
        if tweet_p[1] != []:
            output_emoji = output_emoji + ' ' + ' '.join(tweet_p[1])
        if tweet_p[2] != []:
            output_hash = output_hash + ' ' + ' '.join(tweet_p[2])
        if tweet_p[3] != []:
            output_mention = output_mention + ' ' + ' '.join(tweet_p[3])
    
    df_out.loc[0, 'user_id'] = user_id
    df_out.astype({'user_id': np.int32})
    df_out.loc[0, 'text'] = output_txt
    df_out.loc[0, 'emojis'] = output_emoji
    df_out.loc[0, 'hashtags'] = output_hash
    df_out.loc[0, 'mentions'] = output_mention
    
    return df_out


def tweet_split_2_process_v2(user_id, time, tweets_str):
    '''['podcast', 'time', 'text', 'emojis', 'hashtags', 'mentions']
    '''
    df_out = pd.DataFrame(columns=['podcast'])
    output_txt = ''
    output_emoji = ''
    output_hash = ''
    output_mention = ''
    
    tweets_str = tweets_str.replace('''', "''', "', '")
    tweets_str = tweets_str.replace('''", ''''', "', '")
    tweets_str = tweets_str.replace('''", "''', "', '")
    tweet_list = tweets_str.split("', '")
    
    for tweet in tweet_list:
        tweet_p = my_preprocess(tweet)
        output_txt = output_txt + ' ' + tweet_p[0]
        if tweet_p[1] != []:
            output_emoji = output_emoji + ' ' + ' '.join(tweet_p[1])
        if tweet_p[2] != []:
            output_hash = output_hash + ' ' + ' '.join(tweet_p[2])
        if tweet_p[3] != []:
            output_mention = output_mention + ' ' + ' '.join(tweet_p[3])
    
    df_out.loc[0, 'podcast'] = user_id
    df_out.loc[0, 'time'] = time
    df_out.loc[0, 'text'] = output_txt
    df_out.loc[0, 'emojis'] = output_emoji
    df_out.loc[0, 'hashtags'] = output_hash
    df_out.loc[0, 'mentions'] = output_mention
    
    return df_out


def my_preprocess(text):
    '''
    '''
    doc_emoji = find_emoji(text)
    doc_hash = find_hash(text)
    doc_mentions = find_mention(text)
    text = text.lower()
    text = text.replace('\\n',' ')
    text = p.clean(text)
    text = text.translate(str.maketrans(' ', ' ', punct_str))
    text = re.sub(r' \d+ ', ' ', text)
    text = re.sub(r' \d+ ', ' ', text)
    words = []
    for word in text.split(' '):
        words.append(word)
    words = [w for w in words if not w in stop_words]
    result = []
#     for token in gensim.utils.simple_preprocess(text):
#         if token not in stop_words:  # and len(token) > 3:
#             result.append(token)     # lemmatize_stemming(token))
#     result = result + doc_emoji + doc_hash + doc_mentions
    result = words + doc_emoji + doc_hash + doc_mentions
    tweet_txt = ' '.join(result)
#     return tweet_txt
    return ' '.join(words), doc_emoji, doc_hash, doc_mentions

In [7]:
resource_dir = '/home/ed/github/pod_tweets/resources/'
# pdcst_tweets0 = pd.read_csv('/run/media/ed/SD/follower_tweets/1A.csv')
# pdcst_tweets1 = pd.read_csv('/run/media/ed/SD/follower_tweets/GettingCuriouswithJonathanVanNess.csv')

In [8]:
# X = os.listdir('/run/media/ed/SD/follower_tweets/')
# X = sorted(X)
# X

In [9]:
# X[128:]

In [10]:
# for pdcst in X[129:]:
#     print('Converting {}...'.format(pdcst[:-4]), end='')
#     pdcst_tweet_df = pd.read_csv('/run/media/ed/SD/follower_tweets/' + pdcst)
#     tweet_split_df = pd.DataFrame(columns=['user_id', 'text', 'emojis', 'hashtags', 'mentions'])
#     for x in range(len(pdcst_tweet_df)):
#         if pdcst_tweet_df.loc[x, 'tweets'] != '[]':
#             temp_df = tweet_split_2_process(pdcst_tweet_df.loc[x, 'user_id'], pdcst_tweet_df.loc[x, 'tweets'])
#             tweet_split_df = tweet_split_df.append(temp_df, ignore_index=True)
#     tweet_split_df['podcast_name'] = pdcst[:-4]
#     print('Saving...', end='')
#     tweet_split_df.to_pickle('/run/media/ed/SD/follower_tweets/' + pdcst[:-4] + '.pkl')
#     print('Done')

In [9]:
# tweet_split_df0 = pd.DataFrame(columns=['user_id', 'text', 'emojis', 'hashtags', 'mentions'])

# for x in range(len(pdcst_tweets0)):
#     if pdcst_tweets0.loc[x, 'tweets'] != '[]':
#         temp_df = tweet_split_2_process(pdcst_tweets0.loc[x, 'user_id'], pdcst_tweets0.loc[x, 'tweets'])
#         tweet_split_df0 = tweet_split_df0.append(temp_df, ignore_index=True)

In [10]:
# tweet_split_df1 = pd.DataFrame(columns=['user_id', 'text', 'emojis', 'hashtags', 'mentions'])

# for x in range(len(pdcst_tweets1)):
#     if pdcst_tweets1.loc[x, 'tweets'] != '[]':
#         temp_df = tweet_split_2_process(pdcst_tweets1.loc[x, 'user_id'], pdcst_tweets1.loc[x, 'tweets'])
#         tweet_split_df1 = tweet_split_df1.append(temp_df, ignore_index=True)

In [11]:
# pickle_dir = '/run/media/ed/SD/tweet_pickles/'
# X = os.listdir(pickle_dir)
# X = sorted(X)
# X

# temp_df = pd.read_pickle('/run/media/ed/SD/follower_tweets/' + '1A.pkl')
# temp_df = pd.read_pickle(pickle_dir + X[0])
# all_tweets = temp_df
# for pdcst in X[1:]:
#     temp_df = pd.read_pickle(pickle_dir + pdcst)
#     all_tweets = all_tweets.append(temp_df, ignore_index=True)
# all_tweets = all_tweets.append(pd.read_pickle('/run/media/ed/SD/follower_tweets/' + '1YearDailyAudioBible.pkl'), ignore_index=True)
# all_tweets

In [31]:
# vectorizer = CountVectorizer(encoding='unicode', lowercase=None, strip_accents=None, analyzer='word')
# vectorizer = TfidfVectorizer(encoding='unicode', lowercase=None, strip_accents=None, analyzer='word',
#                              max_df=0.50, min_df = 10)
vectorizer = TfidfVectorizer(
    strip_accents = 'ascii',
    encoding='unicode',
    max_df=0.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False)

startT = time.time()
print(startT)
vect_fit = vectorizer.fit(all_tweets['text'])
b_o_w = vectorizer.transform(all_tweets['text'])
vocab = vect_fit.get_feature_names()
print(len(vocab))
print(time.time()-startT)

1581966479.054959
307220
161.4068558216095


In [32]:
sum_words = b_o_w.sum(axis=0)

In [33]:
words_freq = [(word, sum_words[0, idx]) for word, idx in vect_fit.vocabulary_.items()]

In [34]:
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq[:10]

[('just', 96089.0),
 ('from', 88542.0),
 ('what', 85248.0),
 ('like', 84289.0),
 ('about', 83943.0),
 ('will', 72383.0),
 ('they', 69737.0),
 ('when', 67435.0),
 ('dont', 63858.0),
 ('more', 63262.0)]

In [35]:
# CorEx implementation semi-supervised
# '''https://medium.com/pew-research-center-decoded/
# overcoming-the-limitations-of-topic-models-with-a-semi-supervised-approach-b947374e0455'''

anchors = []
model = ct.Corex(n_hidden=50, seed=42, verbose=1)
model = model.fit(
    b_o_w,
    words=vocab
)

corex, rep size: 50
word counts [57. 31. 18. ... 12. 38. 17.]
[0.095 0.095 0.096 0.096 0.096 0.097 0.098 0.094 0.093 0.097 0.1   0.098 0.094 0.093 0.094 0.095 0.097 0.098 0.095 0.098 0.096 0.094 0.097 0.093 0.095 0.096 0.094 0.094 0.097 0.097 0.097 0.096 0.096
 0.097 0.094 0.098 0.096 0.098 0.096 0.095 0.097 0.097 0.098 0.095 0.104 0.096 0.1   0.095 0.093 0.095]
[1.473 2.111 1.308 1.442 1.544 1.455 1.544 1.607 1.607 1.412 1.728 1.607 1.314 1.319 1.717 1.453 1.494 1.648 1.482 2.221 1.589 1.418 2.559 1.632 1.538 1.634 1.377 1.643 1.508 1.653 1.747 1.425 1.356
 1.488 1.408 2.084 1.548 1.464 1.592 1.703 1.51  1.738 2.238 1.451 2.29  1.6   2.185 1.476 1.628 1.378]
[1.51  5.952 0.676 1.552 1.02  0.989 1.004 3.319 1.408 0.802 2.784 3.535 1.251 1.376 2.938 2.88  1.23  1.55  2.69  9.123 2.307 0.932 7.394 1.98  2.013 2.945 1.076 3.349 1.376 2.26  3.065 0.926 0.962
 0.82  1.468 6.436 2.514 2.365 2.457 2.656 1.175 3.542 4.957 1.135 4.295 1.523 5.463 1.862 2.92  0.795]
[2.199 2.942 0.771 2.524 0.98

[0.741 0.605 0.239 0.255 0.557 0.515 0.363 0.676 0.393 1.647 0.517 0.41  1.929 0.394 5.889 0.263 0.357 0.966 0.279 3.912 2.174 1.919 0.845 0.392 2.952 0.869 0.908 0.307 1.146 0.472 0.724 0.915 0.443
 0.625 0.432 0.673 0.722 1.235 2.681 1.181 0.369 0.63  0.531 0.606 0.616 0.516 0.419 0.655 0.456 0.676]
[0.743 0.612 0.241 0.256 0.555 0.515 0.366 0.68  0.395 1.64  0.52  0.411 1.932 0.399 5.888 0.264 0.358 0.967 0.28  3.907 2.173 1.912 0.845 0.393 2.951 0.865 0.909 0.308 1.153 0.474 0.725 0.926 0.444
 0.627 0.434 0.676 0.723 1.246 2.682 1.191 0.369 0.632 0.533 0.613 0.619 0.518 0.42  0.656 0.458 0.675]
[0.746 0.617 0.242 0.258 0.555 0.515 0.368 0.684 0.397 1.644 0.525 0.411 1.94  0.401 5.889 0.265 0.359 0.963 0.281 3.906 2.173 1.919 0.847 0.394 2.948 0.863 0.91  0.308 1.159 0.477 0.727 0.938 0.444
 0.629 0.436 0.675 0.724 1.248 2.684 1.197 0.37  0.637 0.534 0.619 0.621 0.518 0.424 0.657 0.46  0.675]
[0.748 0.621 0.244 0.258 0.555 0.515 0.37  0.686 0.399 1.657 0.528 0.412 1.947 0.402 5.887 

[0.788 0.638 0.252 0.261 0.554 0.51  0.39  0.728 0.419 1.686 0.534 0.419 2.133 0.421 5.876 0.272 0.371 0.933 0.289 3.892 2.154 1.904 0.891 0.405 2.904 1.006 0.919 0.325 1.255 0.516 0.733 1.083 0.443
 0.636 0.447 0.689 0.748 1.275 2.644 1.284 0.384 0.676 0.563 0.792 0.652 0.539 0.452 0.671 0.477 0.675]
[0.789 0.64  0.251 0.261 0.554 0.51  0.391 0.731 0.42  1.686 0.535 0.419 2.133 0.421 5.876 0.272 0.371 0.935 0.289 3.893 2.154 1.903 0.896 0.405 2.903 1.021 0.919 0.325 1.257 0.517 0.733 1.086 0.443
 0.637 0.447 0.69  0.751 1.277 2.649 1.286 0.385 0.676 0.564 0.792 0.653 0.539 0.452 0.672 0.478 0.675]
[0.789 0.627 0.251 0.261 0.554 0.51  0.391 0.733 0.42  1.686 0.535 0.419 2.133 0.421 5.876 0.272 0.371 0.936 0.288 3.892 2.129 1.903 0.907 0.405 2.903 1.042 0.919 0.325 1.26  0.517 0.733 1.091 0.443
 0.637 0.447 0.689 0.759 1.278 2.649 1.287 0.385 0.677 0.564 0.792 0.653 0.539 0.453 0.672 0.478 0.676]
[0.79  0.627 0.251 0.261 0.554 0.511 0.391 0.735 0.42  1.687 0.535 0.419 2.133 0.424 5.876 

[0.799 0.679 0.251 0.263 0.549 0.512 0.396 0.777 0.422 1.686 0.54  0.413 2.131 0.434 5.876 0.276 0.396 0.935 0.29  3.879 2.142 1.89  0.935 0.409 2.897 1.113 0.921 0.326 1.274 0.52  0.74  1.142 0.436
 0.636 0.444 0.686 0.797 1.283 2.638 1.348 0.391 0.689 0.575 0.799 0.655 0.544 0.474 0.677 0.478 0.676]
[0.799 0.686 0.251 0.263 0.549 0.513 0.396 0.777 0.422 1.686 0.54  0.412 2.131 0.434 5.876 0.276 0.397 0.935 0.289 3.879 2.141 1.89  0.937 0.409 2.897 1.113 0.918 0.326 1.276 0.52  0.74  1.143 0.436
 0.637 0.444 0.686 0.797 1.283 2.638 1.35  0.391 0.69  0.575 0.799 0.655 0.544 0.475 0.677 0.478 0.676]
[0.8   0.691 0.251 0.263 0.549 0.513 0.396 0.776 0.422 1.685 0.54  0.412 2.132 0.435 5.876 0.277 0.398 0.935 0.29  3.877 2.141 1.89  0.937 0.409 2.897 1.113 0.919 0.326 1.279 0.521 0.74  1.144 0.436
 0.637 0.444 0.69  0.797 1.282 2.638 1.351 0.391 0.69  0.575 0.8   0.655 0.544 0.474 0.677 0.478 0.676]
[0.8   0.696 0.251 0.263 0.549 0.514 0.397 0.776 0.422 1.686 0.54  0.412 2.132 0.435 5.876 

[0.804 0.764 0.25  0.262 0.55  0.51  0.397 0.771 0.423 1.696 0.543 0.412 2.135 0.448 5.874 0.278 0.41  0.93  0.288 3.786 2.138 1.888 0.933 0.407 2.896 1.12  0.919 0.325 1.288 0.524 0.743 1.297 0.432
 0.638 0.442 0.696 0.801 1.291 2.629 1.361 0.394 0.694 0.57  0.802 0.653 0.544 0.538 0.667 0.477 0.675]
[0.804 0.764 0.25  0.262 0.55  0.509 0.397 0.772 0.423 1.696 0.543 0.412 2.135 0.448 5.874 0.278 0.411 0.929 0.287 3.782 2.139 1.888 0.933 0.406 2.896 1.12  0.92  0.325 1.289 0.524 0.743 1.304 0.432
 0.638 0.442 0.696 0.801 1.291 2.628 1.362 0.394 0.694 0.57  0.803 0.654 0.544 0.547 0.657 0.477 0.675]
[0.805 0.765 0.25  0.262 0.55  0.51  0.397 0.772 0.423 1.696 0.543 0.412 2.135 0.449 5.874 0.277 0.411 0.93  0.287 3.778 2.141 1.888 0.933 0.406 2.896 1.12  0.92  0.325 1.29  0.524 0.743 1.309 0.432
 0.638 0.442 0.697 0.801 1.291 2.627 1.362 0.394 0.694 0.57  0.803 0.654 0.544 0.549 0.657 0.478 0.675]
[0.805 0.765 0.25  0.262 0.55  0.51  0.397 0.772 0.423 1.696 0.543 0.412 2.135 0.449 5.874 

[0.811 0.767 0.249 0.262 0.549 0.51  0.401 0.778 0.424 1.695 0.552 0.414 2.133 0.452 5.873 0.278 0.417 0.931 0.286 3.665 2.137 1.884 0.938 0.401 2.897 1.124 0.921 0.327 1.323 0.528 0.743 1.454 0.432
 0.636 0.439 0.693 0.811 1.293 2.62  1.387 0.397 0.695 0.573 0.805 0.656 0.541 0.661 0.66  0.482 0.673]
[0.811 0.767 0.249 0.262 0.549 0.51  0.401 0.778 0.424 1.696 0.552 0.414 2.133 0.452 5.873 0.278 0.417 0.931 0.286 3.666 2.137 1.884 0.939 0.401 2.897 1.124 0.921 0.327 1.324 0.529 0.743 1.455 0.432
 0.636 0.439 0.693 0.812 1.293 2.62  1.388 0.398 0.695 0.573 0.805 0.656 0.541 0.664 0.66  0.482 0.673]
[0.811 0.767 0.249 0.262 0.549 0.51  0.401 0.778 0.424 1.697 0.552 0.414 2.133 0.452 5.873 0.278 0.417 0.931 0.286 3.663 2.137 1.884 0.939 0.401 2.897 1.124 0.921 0.327 1.324 0.528 0.743 1.455 0.432
 0.636 0.439 0.692 0.813 1.293 2.62  1.39  0.398 0.695 0.573 0.805 0.656 0.541 0.665 0.66  0.482 0.673]
[0.811 0.767 0.249 0.262 0.548 0.51  0.401 0.778 0.425 1.697 0.552 0.412 2.133 0.453 5.873 

[0.815 0.769 0.248 0.262 0.548 0.51  0.403 0.779 0.425 1.698 0.553 0.408 2.133 0.454 5.873 0.278 0.418 0.931 0.285 3.651 2.137 1.885 0.94  0.401 2.897 1.125 0.921 0.327 1.334 0.528 0.743 1.459 0.432
 0.646 0.438 0.688 0.829 1.291 2.619 1.407 0.399 0.7   0.574 0.805 0.657 0.542 0.665 0.661 0.483 0.673]
[0.814 0.769 0.248 0.262 0.548 0.51  0.403 0.779 0.425 1.698 0.553 0.408 2.133 0.454 5.873 0.278 0.417 0.931 0.286 3.652 2.137 1.885 0.94  0.401 2.897 1.125 0.921 0.327 1.334 0.528 0.743 1.459 0.432
 0.646 0.438 0.688 0.829 1.291 2.619 1.408 0.399 0.7   0.574 0.805 0.657 0.542 0.665 0.661 0.483 0.673]
[0.814 0.769 0.248 0.262 0.548 0.51  0.403 0.78  0.425 1.698 0.553 0.408 2.133 0.454 5.873 0.278 0.417 0.931 0.286 3.651 2.137 1.885 0.94  0.401 2.897 1.125 0.921 0.327 1.334 0.528 0.743 1.459 0.432
 0.646 0.438 0.688 0.829 1.291 2.619 1.408 0.399 0.7   0.574 0.805 0.657 0.542 0.665 0.661 0.483 0.673]
[0.814 0.769 0.248 0.262 0.548 0.51  0.403 0.781 0.425 1.698 0.553 0.408 2.133 0.454 5.873 

[0.815 0.77  0.248 0.262 0.548 0.51  0.404 0.725 0.425 1.697 0.554 0.408 2.133 0.455 5.872 0.278 0.419 0.938 0.285 3.654 2.137 1.885 0.941 0.401 2.896 1.123 0.921 0.326 1.38  0.529 0.745 1.491 0.433
 0.646 0.439 0.688 0.842 1.294 2.625 1.418 0.399 0.7   0.575 0.808 0.658 0.541 0.665 0.662 0.483 0.673]
[0.815 0.77  0.248 0.262 0.548 0.51  0.404 0.723 0.425 1.697 0.554 0.408 2.133 0.455 5.872 0.279 0.419 0.937 0.285 3.654 2.137 1.885 0.942 0.401 2.896 1.123 0.921 0.326 1.383 0.529 0.745 1.492 0.433
 0.646 0.439 0.689 0.842 1.294 2.625 1.418 0.399 0.7   0.574 0.808 0.658 0.541 0.665 0.662 0.483 0.673]
[0.815 0.77  0.249 0.262 0.548 0.51  0.404 0.721 0.425 1.697 0.554 0.408 2.133 0.455 5.872 0.279 0.42  0.937 0.285 3.655 2.137 1.885 0.942 0.401 2.896 1.123 0.921 0.326 1.385 0.529 0.745 1.494 0.433
 0.646 0.439 0.689 0.842 1.294 2.625 1.418 0.399 0.7   0.574 0.808 0.658 0.541 0.665 0.662 0.483 0.673]
[0.815 0.77  0.249 0.262 0.548 0.509 0.404 0.718 0.425 1.697 0.554 0.408 2.133 0.455 5.872 

In [36]:
model.tc

50.73649802876969

In [37]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: para, por, se, mas, como, una, es, un, con, esta
Topic #2: trump, president, bernie, trumps, vote, impeachment, senate, donald, sanders, republicans
Topic #3: du, der, je, und, den, ist, ich, et, nicht, des
Topic #4: witnesses, senators, trial, vindman, bolton, ukraine, impeachment trial, documents, oath, evidence
Topic #5: iowa, caucus, iowa caucus, results, yang, caucuses, new hampshire, hampshire, hai, andrew yang
Topic #6: giveaway, chance win, win, enter, follow, chance, retweet, giving away, winner, giving
Topic #7: kobe, bryant, kobe bryant, gianna, helicopter, gigi, helicopter crash, mamba, bryant family, vanessa
Topic #8: game, bowl, super bowl, team, football, player, players, super, coach, league
Topic #9: state, police, war, justice, violence, rights, news, court, killed, iran
Topic #10: dont, like, know, just, what, because, even, really, feel, doesnt
Topic #11: from, public, data, government, system, tax, research, power, companies, policy
Topic #12: new, check,

In [38]:
from corextopic import vis_topic as vt
vt.vis_rep(model, column_label=vocab, prefix='topic-model-example')

Print topics in text file


In [44]:
# Anchors designed to nudge the model towards measuring specific genres
anchors = [
    ['impeachment'],
    ['trump'],
    ['health'],
    ['broadway'],
    ['ukraine'],
    ['super', 'bowl'],
    ['iowa', 'cacus', 'warren', 'bernie'],
    ['kobe'],
    ['climate', 'change']
]
anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=20, seed=42)
model = model.fit(
    b_o_w,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=3 # Tell the model how much it should rely on the anchors
)

In [45]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: impeachment, trial, witnesses, senate, senators, impeachment trial, trumps, republican, gop, republicans
Topic #2: trump, donald trump, donald, president trump, that trump, administration, trump administration, trump will, democrats, anthem
Topic #3: health, health care, students, new, care, community, education, learn, research, work
Topic #4: broadway, right matters, truth matters, matters decency, sun will, that exposes, republican jordan, exposes mitch, faith optimism, optimism founders
Topic #5: ukraine, president, country, united, states, united states, against, americans, justice, foreign
Topic #6: super, bowl, super bowl, kansas, kansas city, chiefs, missouri, city, city missouri, chiefs play
Topic #7: iowa, bernie, warren, caucus, sanders, democratic, caucuses, des moines, moines, candidate
Topic #8: kobe, bryant, kobe bryant, luther king, martin luther, luther, martin, king jr, gianna, king
Topic #9: change, climate, climate change, will change, change world, eyes e

In [40]:
print('Vect params: , \nCorEx params: , ')

Vect params: , 
CorEx params: , 


In [12]:
all_df = pd.read_csv('/home/ed/github/pod_tweets/all_df.csv', index_col=0)

In [13]:
listCol = all_df.columns.tolist()
for x in ['text', 'emojis', 'hashtags', 'mentions']:
    listCol.remove(x)
all_df[listCol]

,user_id,podcast_name,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
0,1.184987e+18,1A,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.947506e+17,1A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.321216e+07,1A,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.906023e+09,1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.056140e+07,1A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244526,2.550301e+09,SeincastASeinfeldPodcast,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244527,3.673256e+07,SeincastASeinfeldPodcast,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244528,8.368942e+07,SeincastASeinfeldPodcast,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244529,3.139828e+08,SeincastASeinfeldPodcast,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
all_df_grouped = all_df.groupby(['podcast_name']).mean()
all_df_grouped

,user_id,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
podcast_name,,,,,,,,,,,,,,,,,,,,,
1A,2.834669e+17,0.037658,0.323330,0.008826,0.031774,0.477493,0.076493,0.152692,0.264195,0.373345,...,0.035599,0.088555,0.042954,0.036481,0.045013,0.047073,0.036187,0.055899,0.047073,0.021183
1YearDailyAudioBible,7.300818e+16,0.065325,0.039009,0.009288,0.028793,0.098452,0.048916,0.022601,0.071827,0.181424,...,0.013932,0.036842,0.030960,0.015170,0.023529,0.035604,0.024149,0.029412,0.019195,0.010836
2DopeQueens,3.989223e+17,0.028797,0.136102,0.018187,0.027281,0.285238,0.056381,0.096090,0.215520,0.525614,...,0.032131,0.068809,0.050925,0.027281,0.043043,0.047893,0.032131,0.044256,0.030312,0.019097
30For30Podcasts,4.026991e+17,0.081898,0.058499,0.024374,0.058174,0.096198,0.480988,0.032499,0.073773,0.337667,...,0.015600,0.108222,0.032499,0.016250,0.024374,0.036399,0.019825,0.038999,0.017225,0.018200
48Hours,5.228186e+17,0.039254,0.146075,0.013192,0.051802,0.196911,0.117439,0.059846,0.171171,0.346847,...,0.017696,0.063063,0.048584,0.020270,0.033140,0.047297,0.022523,0.039897,0.027349,0.015122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KnowledgeFight,3.286221e+17,0.013326,0.182036,0.018124,0.053305,0.370736,0.033316,0.163913,0.336620,0.591151,...,0.033049,0.071162,0.044243,0.037047,0.039712,0.049574,0.037580,0.052772,0.049307,0.025053
LadyGang,3.660667e+17,0.037073,0.043464,0.017897,0.058166,0.090125,0.082454,0.026526,0.060403,0.453819,...,0.025567,0.055289,0.052093,0.016938,0.032279,0.045382,0.019815,0.034516,0.016938,0.009588
PlanetMoney,1.357657e+17,0.063380,0.183662,0.013239,0.024507,0.422535,0.097465,0.098592,0.247042,0.330986,...,0.041408,0.071268,0.052394,0.030423,0.047606,0.054085,0.047042,0.057465,0.040563,0.018310


In [17]:
listCol.remove('user_id')
listCol.remove('podcast_name')

ValueError: list.remove(x): x not in list

In [18]:
from sklearn.feature_selection import VarianceThreshold
v = 0.003
sel = VarianceThreshold(threshold=(v * (1 - v)))
print(all_df_grouped[listCol].shape)
X_out = sel.fit_transform(all_df_grouped[listCol])
print(X_out.shape)

(70, 100)
(70, 23)


In [23]:
arr = sel.get_support()
for x in range(10):
    print('Topic {}: {}'.format(x + 1, arr[x]))

Topic 1: False
Topic 2: True
Topic 3: False
Topic 4: False
Topic 5: True
Topic 6: True
Topic 7: True
Topic 8: True
Topic 9: True
Topic 10: True


In [24]:
good_topics = []

for x in range(len(arr)):
    if arr[x]:
        good_topics.append(listCol[x])
good_topics        

['topic_2',
 'topic_5',
 'topic_6',
 'topic_7',
 'topic_8',
 'topic_9',
 'topic_10',
 'topic_12',
 'topic_13',
 'topic_14',
 'topic_17',
 'topic_18',
 'topic_23',
 'topic_24',
 'topic_27',
 'topic_30',
 'topic_31',
 'topic_33',
 'topic_43',
 'topic_47',
 'topic_50',
 'topic_58',
 'topic_64']

In [28]:
final_cols = ['podcast_name']
final_cols.extend(good_topics)
final_cols

['podcast_name',
 'topic_2',
 'topic_5',
 'topic_6',
 'topic_7',
 'topic_8',
 'topic_9',
 'topic_10',
 'topic_12',
 'topic_13',
 'topic_14',
 'topic_17',
 'topic_18',
 'topic_23',
 'topic_24',
 'topic_27',
 'topic_30',
 'topic_31',
 'topic_33',
 'topic_43',
 'topic_47',
 'topic_50',
 'topic_58',
 'topic_64']

In [30]:
sim_df = all_df_grouped[good_topics]
sim_df

,topic_2,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_12,topic_13,topic_14,...,topic_24,topic_27,topic_30,topic_31,topic_33,topic_43,topic_47,topic_50,topic_58,topic_64
podcast_name,,,,,,,,,,,,,,,,,,,,,
1A,0.323330,0.477493,0.076493,0.152692,0.264195,0.373345,0.323625,0.396587,0.202118,0.278023,...,0.416887,0.221830,0.339217,0.327449,0.456311,0.294204,0.449250,0.284201,0.239482,0.266549
1YearDailyAudioBible,0.039009,0.098452,0.048916,0.022601,0.071827,0.181424,0.141796,0.252322,0.152322,0.133127,...,0.186068,0.151703,0.142105,0.132508,0.219814,0.259752,0.250464,0.159752,0.153251,0.080805
2DopeQueens,0.136102,0.285238,0.056381,0.096090,0.215520,0.525614,0.318885,0.411943,0.184298,0.189754,...,0.406487,0.227645,0.334041,0.295847,0.431949,0.416793,0.386481,0.216732,0.220370,0.164898
30For30Podcasts,0.058499,0.096198,0.480988,0.032499,0.073773,0.337667,0.101397,0.267468,0.120572,0.080923,...,0.245369,0.161196,0.183945,0.186545,0.340591,0.218395,0.273318,0.324017,0.116672,0.120572
48Hours,0.146075,0.196911,0.117439,0.059846,0.171171,0.346847,0.130631,0.308880,0.159266,0.125483,...,0.264157,0.189833,0.232947,0.268018,0.344595,0.313707,0.313063,0.215573,0.169562,0.133526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KnowledgeFight,0.182036,0.370736,0.033316,0.163913,0.336620,0.591151,0.295043,0.469616,0.134595,0.190299,...,0.440032,0.231077,0.430704,0.409915,0.477079,0.345682,0.411780,0.232409,0.214552,0.136461
LadyGang,0.043464,0.090125,0.082454,0.026526,0.060403,0.453819,0.129115,0.348993,0.171940,0.100671,...,0.267817,0.212528,0.225951,0.197188,0.356663,0.395334,0.291787,0.190476,0.168744,0.092042
PlanetMoney,0.183662,0.422535,0.097465,0.098592,0.247042,0.330986,0.425070,0.331268,0.288732,0.394085,...,0.443662,0.285352,0.305634,0.227606,0.434366,0.269014,0.476056,0.332676,0.292958,0.317183


In [48]:
sim_df.to_csv('topic_percent_matrix.csv')

In [56]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = cosine_similarity(sim_df)
cos_sim_df = pd.DataFrame(cos_sim, columns=sim_df.index)
cos_sim_df = cos_sim_df.T
cos_sim_df.columns = cos_sim_df.index.tolist()
# cs2 = cos_sim_df[[0]].sort_values(by=0, ascending=False)

cos_sim_df.to_csv('cos_sim_df.csv')

cos_sim_df

,1A,1YearDailyAudioBible,2DopeQueens,30For30Podcasts,48Hours,60Minutes,83WeekswithEricBischoff,99Invisible,AccidentalTechPodcast,Accused,...,Invisibilia,JalenJacoby,JennaJulienPodcast,JockoPodcast,JuicyScoopwithHeatherMcDonald,KnowledgeFight,LadyGang,PlanetMoney,PodSaveAmerica,SeincastASeinfeldPodcast
podcast_name,,,,,,,,,,,,,,,,,,,,,
1A,1.000000,0.919313,0.962386,0.820541,0.951630,0.988336,0.871355,0.958022,0.909432,0.938690,...,0.984140,0.810846,0.808020,0.946910,0.922775,0.968369,0.866244,0.980355,0.989875,0.932940
1YearDailyAudioBible,0.919313,1.000000,0.962348,0.869083,0.971750,0.946951,0.937743,0.952218,0.929148,0.956101,...,0.942542,0.872379,0.897273,0.968484,0.967928,0.918221,0.963496,0.924337,0.922064,0.966380
2DopeQueens,0.962386,0.962348,1.000000,0.869263,0.984259,0.970561,0.941537,0.965890,0.921537,0.932792,...,0.972411,0.869484,0.929048,0.978713,0.979938,0.984521,0.957973,0.945936,0.970593,0.969478
30For30Podcasts,0.820541,0.869083,0.869263,1.000000,0.910926,0.886128,0.947483,0.830342,0.796500,0.858145,...,0.817761,0.996539,0.831208,0.929228,0.900980,0.834313,0.899239,0.815180,0.855365,0.933594
48Hours,0.951630,0.971750,0.984259,0.910926,1.000000,0.977367,0.965408,0.938486,0.896418,0.937161,...,0.946009,0.914618,0.920356,0.981322,0.993048,0.964709,0.969253,0.927063,0.971779,0.984508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KnowledgeFight,0.968369,0.918221,0.984521,0.834313,0.964709,0.964635,0.910629,0.942866,0.890607,0.893545,...,0.963006,0.834024,0.898712,0.955180,0.955151,1.000000,0.910781,0.933870,0.979724,0.943234
LadyGang,0.866244,0.963496,0.957973,0.899239,0.969253,0.906832,0.974113,0.900210,0.870168,0.895546,...,0.885273,0.909904,0.966839,0.955330,0.985747,0.910781,1.000000,0.852476,0.896840,0.964724
PlanetMoney,0.980355,0.924337,0.945936,0.815180,0.927063,0.973061,0.860129,0.982058,0.951768,0.972085,...,0.990654,0.797873,0.770314,0.949406,0.892965,0.933870,0.852476,1.000000,0.951116,0.929249


In [42]:
sim_list = cos_sim_df['KnowledgeFight'].sort_values(ascending=False).index.tolist()
print(sim_list[1:6])

['BehindtheBastards', '2DopeQueens', 'HowDidThisGetMade', 'DuncanTrussellFamilyHour', 'DanCarlinsHardcoreHistory']


In [45]:
sim_dict = {}
for pod in cos_sim_df.index.tolist():
    sim_dict[pod] = cos_sim_df[pod].sort_values(ascending=False).index.tolist()[1:]
    
sim_dict

{'1A': ['FreshAir',
  'FiveThirtyEightPolitics',
  'PodSaveAmerica',
  'IntheDark',
  '60Minutes',
  'Invisibilia',
  'DearSugars',
  'InterceptedwithJeremyScahill',
  'EarHustle',
  'PlanetMoney',
  'CommonSensewithDanCarlin',
  'DanCarlinsHardcoreHistory',
  'DirtyJohn',
  'HeresTheThingwithAlecBaldwin',
  'AmericanHistoryTellers',
  'DeathSexMoney',
  'HiddenBrain',
  'AceOnTheHouse',
  'KnowledgeFight',
  'BehindtheBastards',
  'Heavyweight',
  '2DopeQueens',
  'Caliphate',
  'Homecoming',
  '99Invisible',
  'HowDidThisGetMade',
  'AdamCarollaShow',
  '48Hours',
  'AlisonRosenIsYourNewBestFriend',
  'HowIBuiltThiswithGuyRaz',
  'BitchSeshARealHousewivesBreakdown',
  'JockoPodcast',
  'HappierwithGretchenRubin',
  'DuncanTrussellFamilyHour',
  'Accused',
  'ID10TwithChrisHardwick',
  'SeincastASeinfeldPodcast',
  'AstonishingLegends',
  'Cults',
  'ArmchairExpertwithDaxShepard',
  'AnnaFarisIsUnqualified',
  'AtlantaMonster',
  'GettingCuriouswithJonathanVanNess',
  'JuicyScoopwithH

In [46]:
topic_matrix = pd.DataFrame(sim_dict)
topic_matrix

,1A,1YearDailyAudioBible,2DopeQueens,30For30Podcasts,48Hours,60Minutes,83WeekswithEricBischoff,99Invisible,AccidentalTechPodcast,Accused,...,Invisibilia,JalenJacoby,JennaJulienPodcast,JockoPodcast,JuicyScoopwithHeatherMcDonald,KnowledgeFight,LadyGang,PlanetMoney,PodSaveAmerica,SeincastASeinfeldPodcast
0,FreshAir,ArmchairExpertwithDaxShepard,HowDidThisGetMade,JalenJacoby,JuicyScoopwithHeatherMcDonald,IntheDark,ArtofWrestling,Homecoming,HowIBuiltThiswithGuyRaz,DirtyJohn,...,DearSugars,30For30Podcasts,AndThatsWhyWeDrink,SeincastASeinfeldPodcast,GirlonGuywithAishaTyler,BehindtheBastards,CrimeJunkie,HiddenBrain,HeresTheThingwithAlecBaldwin,JockoPodcast
1,FiveThirtyEightPolitics,Cults,BitchSeshARealHousewivesBreakdown,ForePlay,BitchSeshARealHousewivesBreakdown,FreshAir,Bertcastspodcast,HowIBuiltThiswithGuyRaz,Embedded,HowIBuiltThiswithGuyRaz,...,DeathSexMoney,ForePlay,CrimeJunkie,AdamCarollaShow,AnnaFarisIsUnqualified,2DopeQueens,CLUBLIFE,DirtyJohn,FreshAir,AdamCarollaShow
2,PodSaveAmerica,AtlantaMonster,AlisonRosenIsYourNewBestFriend,BingeModeHarryPotter,ArmchairExpertwithDaxShepard,PodSaveAmerica,SeincastASeinfeldPodcast,DeathSexMoney,99Invisible,PlanetMoney,...,EarHustle,BingeModeHarryPotter,CongratulationswithChrisDElia,AmericanHistoryTellers,ArmchairExpertwithDaxShepard,HowDidThisGetMade,ArmchairExpertwithDaxShepard,Invisibilia,AceOnTheHouse,DISGRACELAND
3,IntheDark,AstonishingLegends,BehindtheBastards,83WeekswithEricBischoff,AlisonRosenIsYourNewBestFriend,1A,DISGRACELAND,HappierwithGretchenRubin,Homecoming,HappierwithGretchenRubin,...,HiddenBrain,IAMRAPAPORTSTEREOPODCAST,GuysWeFd,DanCarlinsHardcoreHistory,AtlantaMonster,DuncanTrussellFamilyHour,GirlonGuywithAishaTyler,DeathSexMoney,1A,AtlantaMonster
4,60Minutes,48Hours,GettingCuriouswithJonathanVanNess,IAMRAPAPORTSTEREOPODCAST,AtlantaMonster,CommonSensewithDanCarlin,AtlantaMonster,Heavyweight,HappierwithGretchenRubin,HiddenBrain,...,PlanetMoney,83WeekswithEricBischoff,GettingCuriouswithJonathanVanNess,CommonSensewithDanCarlin,BitchSeshARealHousewivesBreakdown,DanCarlinsHardcoreHistory,AtlantaMonster,HowIBuiltThiswithGuyRaz,60Minutes,ID10TwithChrisHardwick
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,CongratulationswithChrisDElia,JalenJacoby,Caliphate,HiddenBrain,30For30Podcasts,CongratulationswithChrisDElia,PlanetMoney,CongratulationswithChrisDElia,ForePlay,30For30Podcasts,...,CongratulationswithChrisDElia,HiddenBrain,HiddenBrain,JalenJacoby,HiddenBrain,CLUBLIFE,PlanetMoney,AndThatsWhyWeDrink,ForePlay,AccidentalTechPodcast
65,ForePlay,Caliphate,ForePlay,AccidentalTechPodcast,AccidentalTechPodcast,AndThatsWhyWeDrink,HiddenBrain,ForePlay,CongratulationswithChrisDElia,JalenJacoby,...,ForePlay,AccidentalTechPodcast,PlanetMoney,InterceptedwithJeremyScahill,AccidentalTechPodcast,ForePlay,HiddenBrain,30For30Podcasts,30For30Podcasts,JennaJulienPodcast
66,30For30Podcasts,30For30Podcasts,JalenJacoby,InterceptedwithJeremyScahill,InterceptedwithJeremyScahill,JalenJacoby,InterceptedwithJeremyScahill,JennaJulienPodcast,JennaJulienPodcast,AndThatsWhyWeDrink,...,JennaJulienPodcast,InterceptedwithJeremyScahill,InterceptedwithJeremyScahill,JennaJulienPodcast,InterceptedwithJeremyScahill,30For30Podcasts,InterceptedwithJeremyScahill,CongratulationswithChrisDElia,JalenJacoby,InterceptedwithJeremyScahill
67,JalenJacoby,Embedded,30For30Podcasts,Caliphate,Caliphate,Embedded,Caliphate,30For30Podcasts,30For30Podcasts,CongratulationswithChrisDElia,...,30For30Podcasts,Caliphate,Caliphate,Caliphate,Caliphate,JalenJacoby,Caliphate,JalenJacoby,JennaJulienPodcast,Caliphate


In [49]:
topic_matrix.to_csv('similarity_rank_matrix.csv')

In [52]:
rank_matrix = pd.DataFrame()
for x2 in range(len(sim_df)):
    df = pd.DataFrame(sim_df.iloc[x2])
    df = df.sort_values(by=[str(df.columns[0])], ascending=False)
    rank_matrix[df.columns[0]] = df.index.tolist()
rank_matrix

rank_matrix.to_csv('list_of_topics.csv')

In [55]:
cos_sim = pd.read_pickle('/home/ed/github/pod_tweets/resources/' + 'rank_matrix_20200213.pkl')
cos_sim

,1A,1YearDailyAudioBible,2DopeQueens,30For30Podcasts,48Hours,60Minutes,83WeekswithEricBischoff,99Invisible,AccidentalTechPodcast,Accused,...,JalenJacoby,JennaJulienPodcast,JockoPodcast,JudgeJohnHodgman,JuicyScoopwithHeatherMcDonald,KnowledgeFight,LadyGang,PlanetMoney,PodSaveAmerica,SeincastASeinfeldPodcast
1A,1.000000,0.952639,0.973255,0.952814,0.964003,0.968212,0.962001,0.978471,0.971146,0.972541,...,0.958160,0.964614,0.954697,0.956779,0.963875,0.963016,0.975648,0.963856,0.972957,0.967962
1YearDailyAudioBible,0.952639,1.000000,0.936216,0.941753,0.933024,0.938040,0.948078,0.959469,0.937421,0.961043,...,0.931937,0.935923,0.938582,0.947079,0.948116,0.951102,0.942262,0.965432,0.946918,0.946030
2DopeQueens,0.973255,0.936216,1.000000,0.943290,0.967188,0.966444,0.954623,0.962523,0.960665,0.964143,...,0.952181,0.958203,0.954641,0.948887,0.958276,0.953618,0.961886,0.957980,0.959354,0.960592
30For30Podcasts,0.952814,0.941753,0.943290,1.000000,0.944884,0.958068,0.937218,0.956117,0.952148,0.964649,...,0.930345,0.938604,0.950348,0.955419,0.963802,0.950610,0.952712,0.956600,0.951004,0.956257
48Hours,0.964003,0.933024,0.967188,0.944884,1.000000,0.961305,0.943202,0.965509,0.957431,0.964925,...,0.939051,0.947663,0.963424,0.953369,0.950814,0.951272,0.953453,0.957091,0.964160,0.968719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KnowledgeFight,0.963016,0.951102,0.953618,0.950610,0.951272,0.975050,0.960587,0.973957,0.961494,0.967686,...,0.958968,0.955175,0.955483,0.965769,0.948510,1.000000,0.961040,0.977039,0.967149,0.951880
LadyGang,0.975648,0.942262,0.961886,0.952712,0.953453,0.965756,0.958919,0.965436,0.959092,0.956520,...,0.959907,0.966400,0.939610,0.946246,0.947136,0.961040,1.000000,0.951595,0.976262,0.942756
PlanetMoney,0.963856,0.965432,0.957980,0.956600,0.957091,0.974715,0.958770,0.963723,0.962677,0.970530,...,0.958900,0.956331,0.954983,0.965839,0.953563,0.977039,0.951595,1.000000,0.956326,0.957182
PodSaveAmerica,0.972957,0.946918,0.959354,0.951004,0.964160,0.977288,0.950596,0.972026,0.956390,0.965136,...,0.942073,0.949380,0.964949,0.947367,0.950580,0.967149,0.976262,0.956326,1.000000,0.949336


In [61]:
x = 47.7079
x = round(x, 1)
str(x)


'47.7'

In [8]:
podcast_tweets = pd.read_csv(resource_dir + 'test_df.csv', index_col=0)

In [9]:
podcast_tweets

,podcast,time,tweets
0,1A,2020-02-20 16:05:00,"We're live with @LaraMBrownPhD, reviewing last..."
1,1A,2020-02-20 15:54:51,@kramos1841 @JacobMonty1 How are the president...
2,1A,2020-02-20 15:46:18,"@kramos1841 @JacobMonty1 ""Those folks who don'..."
3,1A,2020-02-20 15:42:59,Has Donald Trump done enough to gain more of t...
4,1A,2020-02-20 15:31:56,@cjane87 Read more from @cjane87 here. https:/...
...,...,...,...
36208,ChristinaP,2019-06-14 03:02:12,It's been a while since we've done a solo epis...
36209,ChristinaP,2019-06-11 21:35:33,The best part about public pools is that there...
36210,ChristinaP,2019-06-11 16:36:09,For the mommies 👖👖👖 https://t.co/69o9wQVAcu
36211,ChristinaP,2019-06-11 16:31:53,Yesssss https://t.co/5Sfxjdqvcl


In [26]:
tweet_split_df = pd.DataFrame(columns=['podcast', 'time', 'text', 'emojis', 'hashtags', 'mentions'])
for x in range(len(podcast_tweets)):
    if podcast_tweets.loc[x, 'tweets'] != '[]':
        temp_df = tweet_split_2_process_v2(podcast_tweets.loc[x, 'podcast'],
                                        podcast_tweets.loc[x, 'time'],
                                        podcast_tweets.loc[x, 'tweets'])
        tweet_split_df = tweet_split_df.append(temp_df, ignore_index=True)
tweet_split_df

,podcast,time,text,emojis,hashtags,mentions
0,1A,2020-02-20 16:05:00,were live with reviewing last nights debate ...,,,@LaraMBrownPhD
1,1A,2020-02-20 15:54:51,presidential candidates marketing campaigns l...,,,@kramos1841 @JacobMonty1 @JacobMonty1
2,1A,2020-02-20 15:46:18,folks dont identity with latino heritage more...,,,@kramos1841 @JacobMonty1 @kramos1841
3,1A,2020-02-20 15:42:59,donald trump done enough gain more latino vot...,,,@JacobMonty1
4,1A,2020-02-20 15:31:56,read more from here,,,@cjane87 @cjane87
...,...,...,...,...,...,...
36208,ChristinaP,2019-06-14 03:02:12,been a since weve done a solo episode with gu...,,#YMH,@tomsegura @ChristinaP @J_Potter
36209,ChristinaP,2019-06-11 21:35:33,best part about public pools that always wome...,,#inclusive #BodyPositivity #mistermommy,
36210,ChristinaP,2019-06-11 16:36:09,mommies,👖 👖 👖,,
36211,ChristinaP,2019-06-11 16:31:53,yesssss,,,


In [37]:
tweet_split_df #.to_csv('podcasts_tweets.csv')

,podcast,time,text,emojis,hashtags,mentions
0,1A,2020-02-20 16:05:00,were live with reviewing last nights debate ...,,,@LaraMBrownPhD
1,1A,2020-02-20 15:54:51,presidential candidates marketing campaigns l...,,,@kramos1841 @JacobMonty1 @JacobMonty1
2,1A,2020-02-20 15:46:18,folks dont identity with latino heritage more...,,,@kramos1841 @JacobMonty1 @kramos1841
3,1A,2020-02-20 15:42:59,donald trump done enough gain more latino vot...,,,@JacobMonty1
4,1A,2020-02-20 15:31:56,read more from here,,,@cjane87 @cjane87
...,...,...,...,...,...,...
36208,ChristinaP,2019-06-14 03:02:12,been a since weve done a solo episode with gu...,,#YMH,@tomsegura @ChristinaP @J_Potter
36209,ChristinaP,2019-06-11 21:35:33,best part about public pools that always wome...,,#inclusive #BodyPositivity #mistermommy,
36210,ChristinaP,2019-06-11 16:36:09,mommies,👖 👖 👖,,
36211,ChristinaP,2019-06-11 16:31:53,yesssss,,,


In [50]:
test_df0 = tweet_split_df.groupby('podcast')['text'].apply(' '.join).reset_index()
test_df1 = tweet_split_df.groupby('podcast')['emojis'].apply(' '.join).reset_index()
test_df2 = tweet_split_df.groupby('podcast')['hashtags'].apply(' '.join).reset_index()
test_df3 = tweet_split_df.groupby('podcast')['mentions'].apply(' '.join).reset_index()
test_out = test_df0.set_index('podcast').join(test_df1.set_index('podcast')).join(test_df2.set_index('podcast')).join(test_df3.set_index('podcast'))
test_out

,text,emojis,hashtags,mentions
podcast,,,,
1A,were live with reviewing last nights debate ...,...,...,@LaraMBrownPhD @kramos1841 @JacobMonty1 @Jac...
48hours,a investigation uncovered a loophole allowing...,...,#48Hours #48Hours ...,@CBSNews @ianjameslee @...
99piorg,fraktur often associated with being official ...,❤️ 😁 🤔 🚚 🤯 😻 😻 😻 ⬇...,#CoinCheck #WhoLetTheDogsOut #Wh...,@99piorg @kevincaners @99piorg @radioto...
ARossmann02,i knew this picture would been shared like i ...,😉 🤩 🤩 🤩 🐧 ❤️ 😉 😂 😉 ...,#AccusedPodcast #DavidBocks ...,@enquirer @SharonCoolidge @ARossmann02 @T...
AdamMGrant,entrepreneurs pitching ideas applicants pitch...,...,#DemDebate #DemocraticDebate #WednesdayWisd...,@francescagino @kobe...
...,...,...,...,...
thevanishedpod,please share thank i about regarding that w...,😂 🙄 ❤️ ...,#missingperson #kansas #wichita #veteran #mis...,@slowdawning @slowdawning @slowdawning @...
tiesto,duo didnt know needed until should sad gets...,📸 ❤️ ❤️ ❤️ ❤️ ❤️ ❤️ ❤️ 🙏🏻 😳 💙 🌹 ❤️ 🔥 ...,#Avicii #music #edm #YouShouldBeSad ...,@tiesto @halsey @jameshales20 @tiesto @hals...
travismcelroy,semester break thundermen getting paid a litt...,🤩 😃 💜 ...,#TheZoneCast #VeryGoodJoke #mbmbam ...,@blainecapatch @joshgondelman @joshgond...


In [51]:
test_out.to_csv('podcasts_tweets_condensed.csv')

In [11]:
podcasts_mentions = pd.read_csv('podcasts_tweets_condensed.csv')

In [12]:
podcasts_mentions[['podcast', 'mentions']]

,podcast,mentions
0,1A,@LaraMBrownPhD @kramos1841 @JacobMonty1 @Jac...
1,48hours,@CBSNews @ianjameslee @...
2,99piorg,@99piorg @kevincaners @99piorg @radioto...
3,ARossmann02,@enquirer @SharonCoolidge @ARossmann02 @T...
4,AdamMGrant,@francescagino @kobe...
...,...,...
174,thevanishedpod,@slowdawning @slowdawning @slowdawning @...
175,tiesto,@tiesto @halsey @jameshales20 @tiesto @hals...
176,travismcelroy,@blainecapatch @joshgondelman @joshgond...
177,treygraham,@zlotucha @akakarenwilson @IAmBlankenship ...


In [13]:
pod_recc = podcasts_mentions['podcast'].tolist()

In [14]:
pod_rec = []
for x in pod_recc:
    pod_rec.append('@' + x.lower())
pod_rec[:10]

['@1a',
 '@48hours',
 '@99piorg',
 '@arossmann02',
 '@adammgrant',
 '@alecbaldwin',
 '@alisonrosen',
 '@annakfaris',
 '@astonshnglegnds',
 '@benmandelker']

In [15]:
pod_rec_ment = pd.DataFrame()
pod_rec_ment['recommender'] = pod_rec
pod_rec_ment['mentioned'] = podcasts_mentions['mentions']
pod0ments = pod_rec_ment.loc[0, 'mentioned'].lower().split(' ')

pod_mentions = set(pod0ments)

pod_rec_set = set(pod_rec)
pod_rec_set

rec_ment_df = pd.DataFrame(columns=['recommender', 'mention'])

for x in range(len(pod_rec)):
#     print(pod_rec[x] + ': ', end='')
    pod0ments = set(pod_rec_ment.loc[x, 'mentioned'].lower().split(' '))
    if pod_rec[x] in pod0ments:
        pod0ments.remove(pod_rec[x])
    X = pod_rec_set.intersection(pod0ments)
    if X != []:
        for y in X:
            temp_df = pd.DataFrame({'recommender': [pod_rec[x]], 'mention': [y]})
            rec_ment_df = rec_ment_df.append(temp_df, ignore_index=True)
#     print(list(X))
    
rec_ment_df

,recommender,mention
0,@99piorg,@hrishihirway
1,@99piorg,@thememorypalace
2,@annakfaris,@peteholmes
3,@annakfaris,@jvn
4,@billsimmons,@colincowherd
...,...,...
230,@thevanishedpod,@bobrufftruth
231,@travismcelroy,@justinmcelroy
232,@travismcelroy,@sydneemcelroy
233,@travismcelroy,@matthewmercer


In [16]:
pod_sn_df = pd.read_csv(resource_dir + 'pod-twitter_list2.csv', index_col=0)
pod_sn_df = pod_sn_df[['Name', 'twitter_s_n']]
pod_sn_df['twitter_s_n'] = pod_sn_df['twitter_s_n'].apply(lambda x: '@' + x.lower())
pod_sn_df['Name'] = pod_sn_df['Name'].apply(lambda x: ''.join(ch for ch in x if ch.isalnum()))
pod_sn_df


,Name,twitter_s_n
0,1A,@1a
1,1YearDailyAudioBible,@realbrianhardin
2,2DopeQueens,@dopequeenpheebs
3,30For30Podcasts,@ramonashelburne
4,48Hours,@48hours
...,...,...
182,WorkLifewithAdamGrant,@adammgrant
183,YouMadeItWeirdwithPeteHolmes,@peteholmes
184,YouMustRememberThis,@karinalongworth
185,YoungHouseLoveHasAPodcast,@johnpetersik


In [17]:
sndict = {}
for x in range(len(pod_sn_df)):
    sndict[pod_sn_df.loc[x, 'twitter_s_n']] = pod_sn_df.loc[x, 'Name']
sndict

{'@1a': '1A',
 '@realbrianhardin': '1YearDailyAudioBible',
 '@dopequeenpheebs': '2DopeQueens',
 '@ramonashelburne': '30For30Podcasts',
 '@48hours': '48Hours',
 '@cbsnewsradio': '60Minutes',
 '@heyheyitsconrad': 'SomethingtoWrestlewithBrucePrichard',
 '@99piorg': '99Invisible',
 '@atpfm': 'AccidentalTechPodcast',
 '@arossmann02': 'Accused',
 '@adamcarolla': 'TheAdamandDrDrewShow',
 '@ginagrad': 'AdamCarollaShow',
 '@thejasikanicole': 'AliceIsntDead',
 '@alisonrosen': 'AlisonRosenIsYourNewBestFriend',
 '@lindsayagraham': 'AmericanHistoryTellers',
 '@theemschulz': 'AndThatsWhyWeDrink',
 '@annakfaris': 'AnnaFarisIsUnqualified',
 '@daxshepard': 'ArmchairExpertwithDaxShepard',
 '@coltcabana': 'ArtofWrestling',
 '@astonshnglegnds': 'AstonishingLegends',
 '@paynelindsey': 'UpandVanished',
 '@bastardspod': 'BehindtheBastards',
 '@bertkreischer': 'Bertcastspodcast',
 '@netw3rk': 'BingeModeHarryPotter',
 '@caseyrosewilson': 'BitchSeshARealHousewivesBreakdown',
 '@rcallimachi': 'Caliphate',
 '@tie

In [18]:
print(rec_ment_df.loc[0, 'recommender'])  #, 'recommender']) # [rec_ment_df['recommender'] == '@99piorg']
rename = pod_sn_df[pod_sn_df['twitter_s_n'] == '@1a'].loc[0, 'Name']
rename

@99piorg


'1A'

In [19]:
rec_ment_df2 = pd.DataFrame(columns=['recommender', 'mention'])

for x in range(len(rec_ment_df)):
    rec_sn = rec_ment_df.loc[x, 'recommender']
    men_sn = rec_ment_df.loc[x, 'mention']
    temp_df = pd.DataFrame({'recommender': [sndict[rec_sn]], 'mention': [sndict[men_sn]]})
    rec_ment_df2 = rec_ment_df2.append(temp_df, ignore_index=True)
    
rec_ment_df2

,recommender,mention
0,99Invisible,TheWestWingWeekly
1,99Invisible,thememorypalace
2,AnnaFarisIsUnqualified,YouMadeItWeirdwithPeteHolmes
3,AnnaFarisIsUnqualified,GettingCuriouswithJonathanVanNess
4,TheBillSimmonsPodcast,TheHerdwithColinCowherd
...,...,...
230,TheVanishedPodcast,TruthJusticewithBobRuff
231,TheAdventureZone,MyBrotherMyBrotherAndMe
232,TheAdventureZone,SawbonesAMaritalTourofMisguidedMedicine
233,TheAdventureZone,CriticalRole


In [84]:
test_df = rec_ment_df2.groupby('recommender')['mention'].apply(','.join).reset_index()
search_dict = {}
for x in range(len(test_df)):
    search_dict[test_df.loc[x, 'recommender']] = test_df.loc[x, 'mention'].split(',')
search_dict
joblib.dump(search_dict, resource_dir+'rec-ment_dict.joblib')

['/home/ed/github/pod_tweets/resources/rec-ment_dict.joblib']

In [43]:
test_rankings = pd.DataFrame({'99Invisible': ['TheWestWingWeekly', 'thememorypalace'],
                              'AdamCarollaShow': ['TheAdamandDrDrewShow', 'thememorypalace'],
                              'BingeModeHarryPotter': ['podA', 'podB']})
test_rankings

,99Invisible,AdamCarollaShow,BingeModeHarryPotter
0,TheWestWingWeekly,TheAdamandDrDrewShow,podA
1,thememorypalace,thememorypalace,podB


In [44]:
if test_rankings.loc[0,'99Invisible'] in search_dict['99Invisible']:
    print(1)
else:
    print(1)

1


In [45]:
for x in test_rankings.columns.tolist():
    for y in range(len(test_rankings)):
        if test_rankings.loc[y, x] in search_dict[x]:
            test_rankings.loc[y, x] = 1
        else:
            test_rankings.loc[y, x] = np.nan
test_rankings

,99Invisible,AdamCarollaShow,BingeModeHarryPotter
0,1,1,NaN
1,1,NaN,NaN


In [52]:
test_rankings[:1].mean()

99Invisible             1.0
AdamCarollaShow         1.0
BingeModeHarryPotter    NaN
dtype: float64

In [55]:
cos_sim_df = pd.read_csv(resource_dir + 'cos_sim_df.csv', index_col=0)
cos_sim_df

,1A,1YearDailyAudioBible,2DopeQueens,30For30Podcasts,48Hours,60Minutes,83WeekswithEricBischoff,99Invisible,AccidentalTechPodcast,Accused,...,Invisibilia,JalenJacoby,JennaJulienPodcast,JockoPodcast,JuicyScoopwithHeatherMcDonald,KnowledgeFight,LadyGang,PlanetMoney,PodSaveAmerica,SeincastASeinfeldPodcast
podcast_name,,,,,,,,,,,,,,,,,,,,,
1A,1.000000,0.919313,0.962386,0.820541,0.951630,0.988336,0.871355,0.958022,0.909432,0.938690,...,0.984140,0.810846,0.808020,0.946910,0.922775,0.968369,0.866244,0.980355,0.989875,0.932940
1YearDailyAudioBible,0.919313,1.000000,0.962348,0.869083,0.971750,0.946951,0.937743,0.952218,0.929148,0.956101,...,0.942542,0.872379,0.897273,0.968484,0.967928,0.918221,0.963496,0.924337,0.922064,0.966380
2DopeQueens,0.962386,0.962348,1.000000,0.869263,0.984259,0.970561,0.941537,0.965890,0.921537,0.932792,...,0.972411,0.869484,0.929048,0.978713,0.979938,0.984521,0.957973,0.945936,0.970593,0.969478
30For30Podcasts,0.820541,0.869083,0.869263,1.000000,0.910926,0.886128,0.947483,0.830342,0.796500,0.858145,...,0.817761,0.996539,0.831208,0.929228,0.900980,0.834313,0.899239,0.815180,0.855365,0.933594
48Hours,0.951630,0.971750,0.984259,0.910926,1.000000,0.977367,0.965408,0.938486,0.896418,0.937161,...,0.946009,0.914618,0.920356,0.981322,0.993048,0.964709,0.969253,0.927063,0.971779,0.984508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KnowledgeFight,0.968369,0.918221,0.984521,0.834313,0.964709,0.964635,0.910629,0.942866,0.890607,0.893545,...,0.963006,0.834024,0.898712,0.955180,0.955151,1.000000,0.910781,0.933870,0.979724,0.943234
LadyGang,0.866244,0.963496,0.957973,0.899239,0.969253,0.906832,0.974113,0.900210,0.870168,0.895546,...,0.885273,0.909904,0.966839,0.955330,0.985747,0.910781,1.000000,0.852476,0.896840,0.964724
PlanetMoney,0.980355,0.924337,0.945936,0.815180,0.927063,0.973061,0.860129,0.982058,0.951768,0.972085,...,0.990654,0.797873,0.770314,0.949406,0.892965,0.933870,0.852476,1.000000,0.951116,0.929249


In [78]:
ranking_lists_df = pd.DataFrame()
for x in cos_sim_df.columns.tolist():
    sim_list = cos_sim_df[x].sort_values(ascending=False).index.tolist()
    test_df = pd.DataFrame({x: sim_list[1:]})
    ranking_lists_df = pd.concat([ranking_lists_df, test_df], axis=1)
ranking_lists_df

,1A,1YearDailyAudioBible,2DopeQueens,30For30Podcasts,48Hours,60Minutes,83WeekswithEricBischoff,99Invisible,AccidentalTechPodcast,Accused,...,Invisibilia,JalenJacoby,JennaJulienPodcast,JockoPodcast,JuicyScoopwithHeatherMcDonald,KnowledgeFight,LadyGang,PlanetMoney,PodSaveAmerica,SeincastASeinfeldPodcast
0,FreshAir,ArmchairExpertwithDaxShepard,HowDidThisGetMade,JalenJacoby,JuicyScoopwithHeatherMcDonald,IntheDark,ArtofWrestling,Homecoming,HowIBuiltThiswithGuyRaz,DirtyJohn,...,DearSugars,30For30Podcasts,AndThatsWhyWeDrink,SeincastASeinfeldPodcast,GirlonGuywithAishaTyler,BehindtheBastards,CrimeJunkie,HiddenBrain,HeresTheThingwithAlecBaldwin,JockoPodcast
1,FiveThirtyEightPolitics,Cults,BitchSeshARealHousewivesBreakdown,ForePlay,BitchSeshARealHousewivesBreakdown,FreshAir,Bertcastspodcast,HowIBuiltThiswithGuyRaz,Embedded,HowIBuiltThiswithGuyRaz,...,DeathSexMoney,ForePlay,CrimeJunkie,AdamCarollaShow,AnnaFarisIsUnqualified,2DopeQueens,CLUBLIFE,DirtyJohn,FreshAir,AdamCarollaShow
2,PodSaveAmerica,AtlantaMonster,AlisonRosenIsYourNewBestFriend,BingeModeHarryPotter,ArmchairExpertwithDaxShepard,PodSaveAmerica,SeincastASeinfeldPodcast,DeathSexMoney,99Invisible,PlanetMoney,...,EarHustle,BingeModeHarryPotter,CongratulationswithChrisDElia,AmericanHistoryTellers,ArmchairExpertwithDaxShepard,HowDidThisGetMade,ArmchairExpertwithDaxShepard,Invisibilia,AceOnTheHouse,DISGRACELAND
3,IntheDark,AstonishingLegends,BehindtheBastards,83WeekswithEricBischoff,AlisonRosenIsYourNewBestFriend,1A,DISGRACELAND,HappierwithGretchenRubin,Homecoming,HappierwithGretchenRubin,...,HiddenBrain,IAMRAPAPORTSTEREOPODCAST,GuysWeFd,DanCarlinsHardcoreHistory,AtlantaMonster,DuncanTrussellFamilyHour,GirlonGuywithAishaTyler,DeathSexMoney,1A,AtlantaMonster
4,60Minutes,48Hours,GettingCuriouswithJonathanVanNess,IAMRAPAPORTSTEREOPODCAST,AtlantaMonster,CommonSensewithDanCarlin,AtlantaMonster,Heavyweight,HappierwithGretchenRubin,HiddenBrain,...,PlanetMoney,83WeekswithEricBischoff,GettingCuriouswithJonathanVanNess,CommonSensewithDanCarlin,BitchSeshARealHousewivesBreakdown,DanCarlinsHardcoreHistory,AtlantaMonster,HowIBuiltThiswithGuyRaz,60Minutes,ID10TwithChrisHardwick
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,CongratulationswithChrisDElia,JalenJacoby,Caliphate,HiddenBrain,30For30Podcasts,CongratulationswithChrisDElia,PlanetMoney,CongratulationswithChrisDElia,ForePlay,30For30Podcasts,...,CongratulationswithChrisDElia,HiddenBrain,HiddenBrain,JalenJacoby,HiddenBrain,CLUBLIFE,PlanetMoney,AndThatsWhyWeDrink,ForePlay,AccidentalTechPodcast
65,ForePlay,Caliphate,ForePlay,AccidentalTechPodcast,AccidentalTechPodcast,AndThatsWhyWeDrink,HiddenBrain,ForePlay,CongratulationswithChrisDElia,JalenJacoby,...,ForePlay,AccidentalTechPodcast,PlanetMoney,InterceptedwithJeremyScahill,AccidentalTechPodcast,ForePlay,HiddenBrain,30For30Podcasts,30For30Podcasts,JennaJulienPodcast
66,30For30Podcasts,30For30Podcasts,JalenJacoby,InterceptedwithJeremyScahill,InterceptedwithJeremyScahill,JalenJacoby,InterceptedwithJeremyScahill,JennaJulienPodcast,JennaJulienPodcast,AndThatsWhyWeDrink,...,JennaJulienPodcast,InterceptedwithJeremyScahill,InterceptedwithJeremyScahill,JennaJulienPodcast,InterceptedwithJeremyScahill,30For30Podcasts,InterceptedwithJeremyScahill,CongratulationswithChrisDElia,JalenJacoby,InterceptedwithJeremyScahill
67,JalenJacoby,Embedded,30For30Podcasts,Caliphate,Caliphate,Embedded,Caliphate,30For30Podcasts,30For30Podcasts,CongratulationswithChrisDElia,...,30For30Podcasts,Caliphate,Caliphate,Caliphate,Caliphate,JalenJacoby,Caliphate,JalenJacoby,JennaJulienPodcast,Caliphate


In [79]:
for x in ranking_lists_df.columns.tolist():
    for y in range(len(ranking_lists_df)):
        if x in search_dict.keys():
            if ranking_lists_df.loc[y, x] in search_dict[x]:
                ranking_lists_df.loc[y, x] = 1
            else:
                ranking_lists_df.loc[y, x] = np.nan
        else:
            ranking_lists_df.loc[y, x] = np.nan
ranking_lists_df

,1A,1YearDailyAudioBible,2DopeQueens,30For30Podcasts,48Hours,60Minutes,83WeekswithEricBischoff,99Invisible,AccidentalTechPodcast,Accused,...,Invisibilia,JalenJacoby,JennaJulienPodcast,JockoPodcast,JuicyScoopwithHeatherMcDonald,KnowledgeFight,LadyGang,PlanetMoney,PodSaveAmerica,SeincastASeinfeldPodcast
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
print(ranking_lists_df['99Invisible'])


0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
     ... 
64    NaN
65    NaN
66    NaN
67    NaN
68    NaN
Name: 99Invisible, Length: 69, dtype: object


In [76]:
search_dict.keys()

dict_keys(['99Invisible', 'AdamCarollaShow', 'AmericanHistoryTellers', 'AnnaFarisIsUnqualified', 'ArmchairExpertwithDaxShepard', 'ArtofWrestling', 'BehindtheBastards', 'Bertcastspodcast', 'BingeModeHarryPotter', 'BitchSeshARealHousewivesBreakdown', 'CongratulationswithChrisDElia', 'CriticalRole', 'DISGRACELAND', 'DanCarlinsHardcoreHistory', 'DearSugars', 'DeathSexMoney', 'DirtyJohn', 'DuncanTrussellFamilyHour', 'EarHustle', 'ForePlay', 'GiantBombcast', 'GirlonGuywithAishaTyler', 'GuysWeFd', 'HappierwithGretchenRubin', 'Heavyweight', 'HelloFromTheMagicTavern', 'Homecoming', 'HowDidThisGetMade', 'IAMRAPAPORTSTEREOPODCAST', 'IntheDark', 'Invisibilia', 'JockoPodcast', 'JudgeJohnHodgman', 'JuicyScoopwithHeatherMcDonald', 'KnowledgeFight', 'Lore', 'LouderWithCrowder', 'MyBrotherMyBrotherAndMe', 'MyFavoriteMurderwithKarenKilgariffandGeorgiaHardstark', 'OffTheVinewithKaitlynBristowe', 'OnBeingwithKristaTippett', 'OntheMedia', 'OprahsSuperSoulConversations', 'PennsSundaySchool', 'PodSaveAmerica